In [ ]:
import pandas as pd
from inducedRainbow.spatial import kNNDistance
from utils import nucStitch
from os.path import isfile, join
from skimage.measure import regionprops_table
from random import seed
import numpy as np

In [2]:
df = pd.read_csv("results/inducedRainbow/allClusterSizesRed.csv")
nucleiPath = 'segmentations/inducedRainbow/final/nuclei'

In [4]:
# k-NN of all marked clones
dfMarked1NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked2NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked3NN = pd.DataFrame(columns= ['age', 'aorta','value'])
dfMarked4NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked5NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])

# k-NN of dividing clones
dfDivided1NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided2NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided3NN = pd.DataFrame(columns= ['age', 'aorta','value'])
dfDivided4NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided5NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])

In [ ]:
# negative control: randomly distribute clones
simulationNumber = 1000
for index, row in df.iterrows():
    marked = df.iloc[index, 2:].to_numpy().sum()
    #only take dividing clusters
    divided = df.iloc[index, 3:].to_numpy().sum()
    markedRandomBasis = []
    dividedRandomBasis = []
    age = row["age"].split("P")[-1]
    aorta = row["aorta"]
    #create stitched image from cropped images
    nucImage = nucStitch.fullNucImage16(join(nucleiPath, age, aorta))
    prop_dict = regionprops_table(nucImage, properties = ('label', 'centroid'))
    dfNuc = pd.DataFrame(prop_dict)
    nucIDs = dfNuc['label'].tolist()
    seed(42)
    for _ in range(simulationNumber):
        markedRandomBasis.append(np.asarray(kNNDistance.randomkNNDistances(nucIDs,marked, dfNuc)))
        # only run this analysis if there are more than 7 dividing clones
        if divided > 7:
            dividedRandomBasis.append(np.asarray(kNNDistance.randomkNNDistances(nucIDs,divided, dfNuc)))

    markedRandomTest = np.asarray(kNNDistance.randomkNNDistances(nucIDs,marked, dfNuc))
    # only run this analysis if there are more than 7 dividing clones
    if divided > 7:
        dividedRandomTest = np.asarray(kNNDistance.randomkNNDistances(nucIDs,divided, dfNuc))
    markedRandomBasis = np.asarray(markedRandomBasis)
    markedRandomTest = np.asarray(markedRandomTest)
    dividedRandomBasis = np.asarray(dividedRandomBasis)
    dividedRandomTest = np.asarray(dividedRandomTest)

    dfMarked1NN = pd.concat([dfMarked1NN,pd.DataFrame({"age": age, "aorta": aorta, "value": markedRandomTest[:,1].flatten()/np.mean(markedRandomBasis[:,:,1].flatten()) -1})])
    dfMarked2NN = pd.concat([dfMarked2NN,pd.DataFrame({"age": age, "aorta": aorta, "value": markedRandomTest[:,2].flatten()/np.mean(markedRandomBasis[:,:,2].flatten()) -1})])
    dfMarked3NN = pd.concat([dfMarked3NN,pd.DataFrame({"age": age, "aorta": aorta, "value": markedRandomTest[:,3].flatten()/np.mean(markedRandomBasis[:,:,3].flatten()) -1})])
    dfMarked4NN = pd.concat([dfMarked4NN,pd.DataFrame({"age": age, "aorta": aorta, "value": markedRandomTest[:,4].flatten()/np.mean(markedRandomBasis[:,:,4].flatten()) -1})])
    dfMarked5NN = pd.concat([dfMarked5NN,pd.DataFrame({"age": age, "aorta": aorta, "value": markedRandomTest[:,5].flatten()/np.mean(markedRandomBasis[:,:,5].flatten()) -1})])

    if divided > 7:
        dfDivided1NN = pd.concat([dfDivided1NN,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedRandomTest[:,1].flatten()/np.mean(dividedRandomBasis[:,:,1].flatten()) -1})])
        dfDivided2NN = pd.concat([dfDivided2NN,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedRandomTest[:,2].flatten()/np.mean(dividedRandomBasis[:,:,2].flatten()) -1})])
        dfDivided3NN = pd.concat([dfDivided3NN,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedRandomTest[:,3].flatten()/np.mean(dividedRandomBasis[:,:,3].flatten()) -1})])
        dfDivided4NN = pd.concat([dfDivided4NN,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedRandomTest[:,4].flatten()/np.mean(dividedRandomBasis[:,:,4].flatten()) -1})])
        dfDivided5NN =pd.concat([dfDivided5NN,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedRandomTest[:,5].flatten()/np.mean(dividedRandomBasis[:,:,5].flatten()) -1})])

In [6]:
dfMarked1NN['kNNType'] = "1NN"
dfMarked2NN['kNNType'] = "2NN"
dfMarked3NN['kNNType'] = "3NN"
dfMarked4NN['kNNType'] = "4NN"
dfMarked5NN['kNNType'] = "5NN"
dfMarkedAll = pd.concat([dfMarked1NN, dfMarked2NN, dfMarked3NN, dfMarked4NN, dfMarked5NN])

In [7]:
dfDivided1NN['kNNType'] = "1NN"
dfDivided2NN['kNNType'] = "2NN"
dfDivided3NN['kNNType'] = "3NN"
dfDivided4NN['kNNType'] = "4NN"
dfDivided5NN['kNNType'] = "5NN"
dfDividedAll = pd.concat([dfDivided1NN, dfDivided2NN, dfDivided3NN, dfDivided4NN, dfDivided5NN])

In [11]:
savePath = "results/kNN"
dfMarkedAll.to_csv(join(savePath, "markedNegativeControlkNN1Run_v2.csv"), index=False)
dfDividedAll.to_csv(join(savePath, "dividedNegativeControlkNN1Run_2.csv"), index=False)

In [15]:

#clones are all in the middle of the image
dfMarked1NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked2NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked3NNMid = pd.DataFrame(columns= ['age', 'aorta','value'])
dfMarked4NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked5NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])

dfDivided1NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided2NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided3NNMid = pd.DataFrame(columns= ['age', 'aorta','value'])
dfDivided4NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided5NNMid = pd.DataFrame(columns= ['age', 'aorta', 'value'])

#clones are biased all at the image border

dfMarked1NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked2NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked3NNOut = pd.DataFrame(columns= ['age', 'aorta','value'])
dfMarked4NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked5NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])

dfDivided1NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided2NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided3NNOut = pd.DataFrame(columns= ['age', 'aorta','value'])
dfDivided4NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided5NNOut = pd.DataFrame(columns= ['age', 'aorta', 'value'])

In [ ]:
##clusters only in middle of image (positive control)
##clusters only at outside of image (positive control)

simulationNumber = 1000
for index, row in df.iterrows():
    marked = df.iloc[index, 2:].to_numpy().sum()
    divided = df.iloc[index, 3:].to_numpy().sum()
    markedRandomBasis = []
    dividedRandomBasis = []
    age = row["age"].split("P")[-1]
    aorta = row["aorta"]
    nucImage = nucStitch.fullNucImage16(join(nucleiPath, age, aorta))
    midNucImage = nucStitch.middleNucImage(join(nucleiPath, age, aorta)).astype(int)
    outNucImage = nucStitch.outsideNucImage(join(nucleiPath, age, aorta)).astype(int)
    prop_dict = regionprops_table(nucImage, properties = ('label', 'centroid'))
    dfNuc = pd.DataFrame(prop_dict)
    nucIDs = dfNuc['label'].tolist()
    seed(42)
    for _ in range(simulationNumber):
        markedRandomBasis.append(np.asarray(kNNDistance.randomkNNDistances(nucIDs,marked, dfNuc)))
        if divided > 7:
            dividedRandomBasis.append(np.asarray(kNNDistance.randomkNNDistances(nucIDs,divided, dfNuc)))

    prop_dictMid = regionprops_table(midNucImage, properties = ('label', 'centroid'))
    dfNucMid = pd.DataFrame(prop_dictMid)
    nucIDsMid = dfNucMid['label'].tolist()
    markedMid = np.asarray(kNNDistance.randomkNNDistances(nucIDsMid,marked, dfNucMid))
    if divided > 7:
        dividedMid = np.asarray(kNNDistance.randomkNNDistances(nucIDsMid,divided, dfNucMid))


    prop_dictOut = regionprops_table(outNucImage, properties = ('label', 'centroid'))
    dfNucOut = pd.DataFrame(prop_dictOut)
    nucIDsOut = dfNucOut['label'].tolist()
    markedOut = np.asarray(kNNDistance.randomkNNDistances(nucIDsOut,marked, dfNucOut))
    if divided > 7:
        dividedOut = np.asarray(kNNDistance.randomkNNDistances(nucIDsOut,divided, dfNucOut))



    markedRandomBasis = np.asarray(markedRandomBasis)
    markedMid = np.asarray(markedMid)
    markedOut = np.asarray(markedOut)
    dividedRandomBasis = np.asarray(dividedRandomBasis)
    dividedMid = np.asarray(dividedMid)
    dividedOut = np.asarray(dividedOut)

    dfMarked1NNMid = pd.concat([dfMarked1NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": markedMid[:,1].flatten()/np.mean(markedRandomBasis[:,:,1].flatten()) -1})])
    dfMarked2NNMid = pd.concat([dfMarked2NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": markedMid[:,2].flatten()/np.mean(markedRandomBasis[:,:,2].flatten()) -1})])
    dfMarked3NNMid = pd.concat([dfMarked3NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": markedMid[:,3].flatten()/np.mean(markedRandomBasis[:,:,3].flatten()) -1})])
    dfMarked4NNMid = pd.concat([dfMarked4NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": markedMid[:,4].flatten()/np.mean(markedRandomBasis[:,:,4].flatten()) -1})])
    dfMarked5NNMid = pd.concat([dfMarked5NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": markedMid[:,5].flatten()/np.mean(markedRandomBasis[:,:,5].flatten()) -1})])

    dfMarked1NNOut = pd.concat([dfMarked1NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": markedOut[:,1].flatten()/np.mean(markedRandomBasis[:,:,1].flatten()) -1})])
    dfMarked2NNOut = pd.concat([dfMarked2NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": markedOut[:,2].flatten()/np.mean(markedRandomBasis[:,:,2].flatten()) -1})])
    dfMarked3NNOut = pd.concat([dfMarked3NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": markedOut[:,3].flatten()/np.mean(markedRandomBasis[:,:,3].flatten()) -1})])
    dfMarked4NNOut = pd.concat([dfMarked4NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": markedOut[:,4].flatten()/np.mean(markedRandomBasis[:,:,4].flatten()) -1})])
    dfMarked5NNOut = pd.concat([dfMarked5NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": markedOut[:,5].flatten()/np.mean(markedRandomBasis[:,:,5].flatten()) -1})])

    if divided > 7:
        dfDivided1NNMid = pd.concat([dfDivided1NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedMid[:,1].flatten()/np.mean(dividedRandomBasis[:,:,1].flatten()) -1})])
        dfDivided2NNMid = pd.concat([dfDivided2NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedMid[:,2].flatten()/np.mean(dividedRandomBasis[:,:,2].flatten()) -1})])
        dfDivided3NNMid = pd.concat([dfDivided3NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedMid[:,3].flatten()/np.mean(dividedRandomBasis[:,:,3].flatten()) -1})])
        dfDivided4NNMid = pd.concat([dfDivided4NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedMid[:,4].flatten()/np.mean(dividedRandomBasis[:,:,4].flatten()) -1})])
        dfDivided5NNMid = pd.concat([dfDivided5NNMid,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedMid[:,5].flatten()/np.mean(dividedRandomBasis[:,:,5].flatten()) -1})])

        dfDivided1NNOut = pd.concat([dfDivided1NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedOut[:,1].flatten()/np.mean(dividedRandomBasis[:,:,1].flatten()) -1})])
        dfDivided2NNOut = pd.concat([dfDivided2NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedOut[:,2].flatten()/np.mean(dividedRandomBasis[:,:,2].flatten()) -1})])
        dfDivided3NNOut = pd.concat([dfDivided3NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedOut[:,3].flatten()/np.mean(dividedRandomBasis[:,:,3].flatten()) -1})])
        dfDivided4NNOut = pd.concat([dfDivided4NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedOut[:,4].flatten()/np.mean(dividedRandomBasis[:,:,4].flatten()) -1})])
        dfDivided5NNOut = pd.concat([dfDivided5NNOut,pd.DataFrame({"age": age, "aorta": aorta, "value": dividedOut[:,5].flatten()/np.mean(dividedRandomBasis[:,:,5].flatten()) -1})])


In [17]:
dfMarked1NNMid['kNNType'] = "1NN"
dfMarked2NNMid['kNNType'] = "2NN"
dfMarked3NNMid['kNNType'] = "3NN"
dfMarked4NNMid['kNNType'] = "4NN"
dfMarked5NNMid['kNNType'] = "5NN"
dfMarkedAllMid = pd.concat([dfMarked1NNMid, dfMarked2NNMid, dfMarked3NNMid, dfMarked4NNMid, dfMarked5NNMid])

dfMarked1NNOut['kNNType'] = "1NN"
dfMarked2NNOut['kNNType'] = "2NN"
dfMarked3NNOut['kNNType'] = "3NN"
dfMarked4NNOut['kNNType'] = "4NN"
dfMarked5NNOut['kNNType'] = "5NN"
dfMarkedAllOut = pd.concat([dfMarked1NNOut, dfMarked2NNOut, dfMarked3NNOut, dfMarked4NNOut, dfMarked5NNOut])

In [18]:
dfDivided1NNMid['kNNType'] = "1NN"
dfDivided2NNMid['kNNType'] = "2NN"
dfDivided3NNMid['kNNType'] = "3NN"
dfDivided4NNMid['kNNType'] = "4NN"
dfDivided5NNMid['kNNType'] = "5NN"
dfDividedAllMid = pd.concat([dfDivided1NNMid, dfDivided2NNMid, dfDivided3NNMid, dfDivided4NNMid, dfDivided5NNMid])

dfDivided1NNOut['kNNType'] = "1NN"
dfDivided2NNOut['kNNType'] = "2NN"
dfDivided3NNOut['kNNType'] = "3NN"
dfDivided4NNOut['kNNType'] = "4NN"
dfDivided5NNOut['kNNType'] = "5NN"
dfDividedAllOut = pd.concat([dfDivided1NNOut, dfDivided2NNOut, dfDivided3NNOut, dfDivided4NNOut, dfDivided5NNOut])

In [24]:
savePath = "/Users/jones/Downloads"
dfMarkedAllMid.to_csv(join(savePath, "markedPositiveControlMidkNN1Run.csv"), index=False)
dfMarkedAllOut.to_csv(join(savePath, "markedPositiveControlOutkNN1Run.csv"), index=False)
dfDividedAllMid.to_csv(join(savePath, "dividedPositiveControlMidkNN1Run.csv"), index=False)
dfDividedAllOut.to_csv(join(savePath, "dividedPositiveControlOutkNN1Run.csv"), index=False)